In [1]:
import numpy as np
import pandas as pd
import os
import sys
import re
import nltk
import string
from dotenv import load_dotenv
from lyricsgenius import Genius
from tqdm.auto import tqdm
load_dotenv()
genius_token = os.getenv('GENIUS_ACCESS_TOKEN')
genius = Genius(genius_token)
import json

In [3]:
# load pickle file
artists_dict = pd.read_pickle('../dict_featuresALL-modified.pickle')

FileNotFoundError: [Errno 2] No such file or directory: '../dict_featuresALL-modified.pickle'

In [4]:
df = pd.read_csv('../nodes.csv')

In [5]:
artist_list = df.iloc[:,1].to_numpy()
artist_list

array(['Post Malone', 'Roddy Ricch', 'Doja Cat', ..., 'Jarryd James',
       'Rebel Wilson', 'Purity Ring'], dtype=object)

In [ ]:
genius_artist = genius.search_artist('Doja Cat', max_songs=3, sort="popularity", allow_name_change=True)


In [ ]:
genius_artist.songs[0].lyrics

In [11]:
files = os.listdir()
done_list = artist_list
files = [f.split('.')[0] for f in files if f.endswith('.json')]
remaining = np.setdiff1d(done_list, files)
print(len(remaining))

152


In [11]:
import multiprocessing as mp

def get_artist(artist):
    try:
        genius_artist = genius.search_artist(artist, max_songs=3, sort="popularity", allow_name_change=True)
        genius_artist.save_lyrics(f'{artist}', verbose=False, overwrite=True)
    
    except Exception as e:
        print(e)

with mp.Pool(mp.cpu_count()) as pool:
    results = list(tqdm(pool.imap(get_artist, remaining), total=len(remaining)))    

Searching for songs by Ruth La’Ontra...
Searching for songs by Dreamville, J. Cole & Lute...
Searching for songs by Cheat Codes & Kris Kross Amsterdam...
Searching for songs by Oscar Maydon & Natanael Cano...
Searching for songs by Bad Bunny & Tainy...
Searching for songs by DJ Luian...
Searching for songs by Leslie Odom, Jr., Anthony Ramos, Daveed Diggs, Okieriete Onaodowan, Lin-Manuel Miranda, Phillipa Soo, Christopher Jackson & Original Broadway Cast of “Hamilton”...
Searching for songs by Beyonca Young...
Searching for songs by 88rising, Joji, Rich Brian & AUGUST 08...




  0%|          | 0/45 [00:00<?, ?it/s]


Searching for songs by Disney Character Voices Inc...
Searching for songs by Grupo Frontera...
Searching for songs by Ziv Zaifman...









Song 1: "Porte Exuberante"
Song 1: "Kingdom (Radio Edit Lyrics)"
Song 1: "Sex"
Done. Found 1 songs.
Searching for songs by Pedro Caparros López...

Done. Found 0 songs.
Searching for songs by Carolina Gaitán, Mauro Castillo, Adassa, Rhenzy Feliz, Diane Guerrero & Stephanie Beatriz...

Song 2: "Kilos De H (En Vivo)"
Done. Found 2 songs.
Searching for songs by Ziv Zaifman, Hugh Jackman & Michelle Williams...

Song 2: "I Got You (Live)"
Done. Found 2 songs.
Searching for songs by Rosalba Alfano...

Song 1: "Midsummer Madness"
Done. Found 1 songs.
Searching for songs by Leslie Odom, Jr., Anthony Ramos, Daveed Diggs, Okieriete Onaodowan, Lin-Manuel Miranda, Phillipa Soo, Christopher Jackson & Original Broadway Cast of “Hamilton”...

Song 1: "Callaita"
Song 1: "Under the Sun"
Song 2: "Callaita (Versión Limpia)"
Song 1: "A Million Dreams"
Done. Found 1

KeyboardInterrupt: 

In [6]:
files = [f for f in os.listdir() if f.endswith('.json')]
len(files)

1123

In [20]:
def get_song_lyrics(artist_json):
    artist = json.load(open(artist_json))
    artist_name = artist['name']
    lyrics = []
    for song in artist['songs']:
        lyrics.append(song['lyrics'])
    return artist_name, lyrics

In [21]:
names_all, lyrics_all = [], []
for f in files:
    artist_name, lyrics = get_song_lyrics(f)
    names_all.append(artist_name)
    lyrics_all.append(lyrics)

In [22]:
df1 = pd.DataFrame(zip(names_all, lyrics_all), columns=['artist', 'lyrics'])

In [23]:
df1['len_lyrics'] = df1.lyrics.apply(lambda x: len(x))
filt = df1['len_lyrics'] != 0
df1 = df1[filt]
print(df1.shape)
df1.head()

(1114, 3)


,artist,lyrics,len_lyrics
0,Marc E. Bassy,[33 ContributorsYou & Me Lyrics[Verse 1: Marc ...,3
1,MisterWives,[20 ContributorsReflections Lyrics[Verse 1]\nY...,3
2,YBN Nahmir,[185 ContributorsRubbin off the Paint Lyrics[I...,3
3,Jeremy Zucker,[41 ContributorsTranslationsEspañol​comethru L...,3
4,Jessica Darrow,[110 ContributorsTranslationsTagalog / ᜆᜄᜎᜓᜄ᜔S...,3


In [24]:
print(df1.lyrics[0][0].strip().split('Lyrics[')[1].replace('\n', ' '))

Verse 1: Marc E. Bassy] Girl you party all the time Don't let me stay on your mind Adderall and cheap wine Just to stay awake in conversation We were always so damn insecure So how could we ever know for sure And disregard the way I know we feel That would make this city way too real  [Chorus: Marc E. Bassy] If we bump into each other On a crowded street It’s not us no more It’s just you and me We’re just strangers in passing casually It’s not us no more It’s just you and me  [Verse 2: Marc E. Bassy] The shirt on my back with the rips Gems in my golden necklace You didn't give me none of this There's nothing for me to miss We were too young on the day we met How could we say we would not forget And in the shadows there's a place for us Somewhere hidden they can't find our love Don't let 'em know You might also like[Chorus: Marc E. Bassy] If we bump into each other On a crowded street It’s not us no more It’s just you and me We’re just strangers in passing casually It’s not us no more I

In [25]:
df1['processed'] = df1['lyrics'].apply(lambda x : ' '.join(x))
df1['processed'] = df1['processed'].apply(lambda x: re.sub("[\(\[].*?[\)\]]", "", x).replace('\n', ' ').split(' Lyrics'))

In [26]:
df1['len_processed'] = df1['processed'].apply(lambda x: len(x))

In [28]:
df1[df1['len_processed'] < 2]
df1 = df1.drop([408, 651])

In [29]:
df1[df1['len_processed'] < 2]

,artist,lyrics,len_lyrics,processed,len_processed


In [30]:
df1['processed'] = df1['processed'].apply(lambda x: x[1])

In [31]:
df2 = df1[['artist', 'processed']]
genres = pd.read_pickle('../dict_genresPreProcessed.pickle')
genres2 = {}
for k, v in genres.items():
    name = k[0]
    genres2[name] = v['top genre']

l1 = [artist for artist in df2.artist if artist not in genres2.keys()]

df2['genre'] = df2['artist'].apply(lambda x: genres2[x] if x in genres2.keys() else None)
df2.dropna(inplace=True)
df2

/tmp/ipykernel_5443/3079943749.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['genre'] = df2['artist'].apply(lambda x: genres2[x] if x in genres2.keys() else None)
/tmp/ipykernel_5443/3079943749.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.dropna(inplace=True)


,artist,processed,genre
0,Marc E. Bassy,Girl you party all the time Don't let me stay...,rap
1,MisterWives,"You didn't close the door Left a crack open, ...",indie
2,YBN Nahmir,That's what I thought you said Now let me off...,trap
3,Jeremy Zucker,I might lose my mind Waking when the sun's do...,alt-z
4,Jessica Darrow,"I'm the strong one, I'm not nervous I'm as to...",movie
...,...,...,...
1117,Rascal Flatts,"Woo Hmm, yeah Life's like a road that you t...",country
1118,Kristen Bell,"I've seen dark before, but not like this This...",movie
1119,StaySolidRocky,"Girls just wanna have fun So they party, so t...",rap
1121,Polo G,"Uh , copped a BMW, new deposit, I picked u...",rap


In [35]:
stop_words = [re.sub('[^a-z_-]', '', stop_word) for stop_word in stop_words]

In [40]:
def tokenizer(input_text):
    stop_words = nltk.corpus.stopwords.words('english')
    stop_words = [re.sub('[^a-z_-]', '', stop_word) for stop_word in stop_words]
    input_text = input_text.lower()
    input_text = re.sub('[^a-z_-]', ' ', input_text)
    tokens = nltk.tokenize.word_tokenize(input_text)
    tokens = [token for token in tokens if token not in stop_words and len(token) > 1]

    return tokens

In [41]:
df2['tokens'] = df2['processed'].apply(lambda x: tokenizer(x))

/tmp/ipykernel_5443/498289619.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['tokens'] = df2['processed'].apply(lambda x: tokenizer(x))


In [42]:
df2

,artist,processed,genre,tokens
0,Marc E. Bassy,Girl you party all the time Don't let me stay...,rap,"[girl, party, time, let, stay, mind, adderall,..."
1,MisterWives,"You didn't close the door Left a crack open, ...",indie,"[close, door, left, crack, open, ignore, faint..."
2,YBN Nahmir,That's what I thought you said Now let me off...,trap,"[thought, said, let, offer, rebuttal, ayy, ayy..."
3,Jeremy Zucker,I might lose my mind Waking when the sun's do...,alt-z,"[might, lose, mind, waking, sun, riding, highs..."
4,Jessica Darrow,"I'm the strong one, I'm not nervous I'm as to...",movie,"[strong, one, nervous, tough, crust, earth, mo..."
...,...,...,...,...
1117,Rascal Flatts,"Woo Hmm, yeah Life's like a road that you t...",country,"[woo, hmm, yeah, life, like, road, travel, one..."
1118,Kristen Bell,"I've seen dark before, but not like this This...",movie,"[seen, dark, like, cold, empty, numb, life, kn..."
1119,StaySolidRocky,"Girls just wanna have fun So they party, so t...",rap,"[girls, wan, na, fun, party, party, lil, mama,..."
1121,Polo G,"Uh , copped a BMW, new deposit, I picked u...",rap,"[uh, copped, bmw, new, deposit, picked, anothe..."


In [43]:
df2['genre'] = df2['artist'].apply(lambda x: genres2[x] if x in genres2.keys() else None)
df2.dropna(inplace=True)
df2

/tmp/ipykernel_5443/1629101791.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['genre'] = df2['artist'].apply(lambda x: genres2[x] if x in genres2.keys() else None)
/tmp/ipykernel_5443/1629101791.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.dropna(inplace=True)


,artist,processed,genre,tokens
0,Marc E. Bassy,Girl you party all the time Don't let me stay...,rap,"[girl, party, time, let, stay, mind, adderall,..."
1,MisterWives,"You didn't close the door Left a crack open, ...",indie,"[close, door, left, crack, open, ignore, faint..."
2,YBN Nahmir,That's what I thought you said Now let me off...,trap,"[thought, said, let, offer, rebuttal, ayy, ayy..."
3,Jeremy Zucker,I might lose my mind Waking when the sun's do...,alt-z,"[might, lose, mind, waking, sun, riding, highs..."
4,Jessica Darrow,"I'm the strong one, I'm not nervous I'm as to...",movie,"[strong, one, nervous, tough, crust, earth, mo..."
...,...,...,...,...
1117,Rascal Flatts,"Woo Hmm, yeah Life's like a road that you t...",country,"[woo, hmm, yeah, life, like, road, travel, one..."
1118,Kristen Bell,"I've seen dark before, but not like this This...",movie,"[seen, dark, like, cold, empty, numb, life, kn..."
1119,StaySolidRocky,"Girls just wanna have fun So they party, so t...",rap,"[girls, wan, na, fun, party, party, lil, mama,..."
1121,Polo G,"Uh , copped a BMW, new deposit, I picked u...",rap,"[uh, copped, bmw, new, deposit, picked, anothe..."


In [44]:
tok_genre = df2.groupby('genre').apply(lambda x: x['tokens'].sum())
tokens_dict = {k: v for k,v in tok_genre.items()}


In [45]:
def TermFrequency(tokens):
    tf_dict = {}
    for token in tokens:
        if token in tf_dict:
            tf_dict[token] += 1
        else:
            tf_dict[token] = 1
    return tf_dict

def idf(tf_dict):
    idf_dict = {}
    N = len(tf_dict)
    all_tokens = set([token for tokens in tf_dict.values() for token in tokens])

    for token in all_tokens:
        n = 0
        for tokens in tf_dict.values():
            if token in tokens:
                n += 1
        idf_dict[token] = np.log(N / n)

    return idf_dict

def tf_idf(tf_dict, idf_dict):
    tf_idf_dict = {}

    for community, tokens in tf_dict.items():
        tf_idf_dict[community] = {}
        for token in tokens:
            tf_idf_dict[community][token] = tokens[token] * idf_dict[token]

    return tf_idf_dict

In [47]:
genres_tf = {}
for genre in tokens_dict.keys():
    genres_tf[genre] = TermFrequency(tokens_dict[genre])
    sort5 = sorted(genres_tf[genre].items(), reverse=True, key=lambda x: x[1])
    print(f'Community {genre} has top 5 words: {sort5[:5]}')

Community a-cappella has top 5 words: [('run', 19), ('girls', 16), ('flashlight', 13), ('cause', 11), ('world', 10)]
Community adult has top 5 words: [('going', 4), ('moon', 3), ('river', 3), ('dream', 2), ('maker', 2)]
Community afrofuturism has top 5 words: [('babe', 12), ('give', 6), ('girl', 6), ('darling', 6), ('bad', 4)]
Community alt has top 5 words: [('guess', 6), ('thought', 5), ('one', 4), ('people', 4), ('know', 3)]
Community alt-z has top 5 words: [('la', 82), ('like', 63), ('know', 57), ('let', 46), ('need', 44)]
Community alt-zernative has top 5 words: [('yeah', 31), ('like', 30), ('na', 27), ('call', 24), ('get', 23)]
Community anime has top 5 words: [('money', 17), ('dolla', 10), ('bills', 8), ('droppin', 8), ('dun', 8)]
Community aussietronica has top 5 words: [('oh', 16), ('call', 15), ('know', 8), ('need', 8), ('somebody', 5)]
Community bachata has top 5 words: [('es', 24), ('que', 15), ('amor', 14), ('te', 11), ('tu', 10)]
Community bass has top 5 words: [('light', 

In [48]:
genres_idf = idf(tokens_dict)

In [49]:
print(sorted(genres_idf.items(), reverse=True, key=lambda x: x[1])[:5])

[('bends', 4.543294782270004), ('contributorsscars', 4.543294782270004), ('elbow', 4.543294782270004), ('gerardo', 4.543294782270004), ('a-carryin', 4.543294782270004)]


In [50]:
genres_tf_idf = tf_idf(genres_tf, genres_idf)

In [54]:
for genres in genres_tf_idf.keys():
    sort5 = sorted(genres_tf_idf[genres].items(), reverse=True, key=lambda x: x[1])
    print(f'{genres} has top 5 words: {sort5[:5]}')

a-cappella has top 5 words: [('flashlight', 50.05191882223076), ('girls', 25.581692849657017), ('motha', 22.71647391135002), ('run', 21.076843977912294), ('belong', 15.584307799288144)]
adult has top 5 words: [('huckleberry', 9.086589564540008), ('breaker', 7.700295203420117), ('river', 6.722129067827876), ('maker', 6.314000842300227), ('moon', 6.175164397446011)]
afrofuturism has top 5 words: [('babe', 19.83507629248607), ('darling', 11.870072548850803), ('linger', 9.086589564540008), ('belong', 7.792153899644072), ('girl', 5.927680324683541)]
alt has top 5 words: [('guess', 10.624236360181335), ('conscious', 7.700295203420117), ('apologies', 6.8893649872037885), ('communicate', 6.8893649872037885), ('thought', 6.826204759610292)]
alt-z has top 5 words: [('la', 126.90012571471307), ('dam', 124.0085697696682), ('la-la-la-la', 61.602361627360935), ('stupid', 53.31864867433286), ('bitter', 44.57533389374191)]
alt-zernative has top 5 words: [('paid', 31.168615598576288), ('em', 28.7794044